In [ ]:
%pip install tensorflow==2.7.0 tensorflow-gpu==2.7.0 opencv-python mediapipe sklearn matplotlib

In [ ]:
import numpy as np
import cv2
import os
import time
import mediapipe as mp

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
def mediapipe_detection(input_image, model):
    # Convert image from BGR to RGB
    input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
    # Make prediction
    detection_results = model.process(input_image)
    # Convert image back to BGR
    input_image = cv2.cvtColor(input_image, cv2.COLOR_RGB2BGR)

    return input_image, detection_results

In [ ]:
def draw_landmarks(input_image, detection_results):
    # Draw face connections
    mp_drawing.draw_landmarks(input_image, detection_results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    
    # Draw pose connections
    mp_drawing.draw_landmarks(input_image, detection_results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    
    # Draw left hand connections
    mp_drawing.draw_landmarks(input_image, detection_results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    
    # Draw right hand connections
    mp_drawing.draw_landmarks(input_image, detection_results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [ ]:
def draw_styled_landmarks(input_image, detection_results):
    # Set drawing specifications for face connections
    face_points = mp_drawing.DrawingSpec(color=(255,0,0), thickness=1, circle_radius=1) 
    face_lines = mp_drawing.DrawingSpec(color=(255,255,0), thickness=1, circle_radius=1)

    # Set drawing specifications for pose connections
    pose_points = mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=4)
    pose_lines = mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2)

    # Set drawing specifications for left hand connections
    left_hand_points = mp_drawing.DrawingSpec(color=(0,150,255), thickness=2, circle_radius=4)
    left_hand_lines = mp_drawing.DrawingSpec(color=(20,20,255), thickness=2, circle_radius=2)

    # Set drawing specifications for right hand connections
    right_hand_points = mp_drawing.DrawingSpec(color=(245,10,30), thickness=2, circle_radius=4)
    right_hand_lines = mp_drawing.DrawingSpec(color=(245,30,240), thickness=2, circle_radius=2)

    # Draw face connections
    mp_drawing.draw_landmarks(input_image, detection_results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, face_points, face_lines)

    # Draw pose connections
    mp_drawing.draw_landmarks(input_image, detection_results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, pose_points, pose_lines)

    # Draw left hand connections
    mp_drawing.draw_landmarks(input_image, detection_results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, left_hand_points, left_hand_lines)

    # Draw right hand connections
    mp_drawing.draw_landmarks(input_image, detection_results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, right_hand_points, right_hand_lines)


In [ ]:
# Initialize video capture
capture = cv2.VideoCapture(0)

# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # Loop while video capture is open
    while capture.isOpened():
        # Read frame from video capture
        ret, image_frame = capture.read()

        # Process frame using mediapipe model
        input_image, detection_results = mediapipe_detection(image_frame, holistic)

        # Draw styled landmarks on frame
        draw_styled_landmarks(input_image, detection_results)

        # Display frame
        cv2.imshow('OpenCV Feed', input_image)

        # Break loop if 'q' key is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # Release video capture and close windows
    capture.release()
    cv2.destroyAllWindows()


In [ ]:
draw_landmarks(image_frame, detection_results)